# TFLite API 서버 배포

### 1단계 - 모델을 .tflite 형식으로 변환

In [ ]:
import tensorflow as tf

# 기존 학습된 모델 로드
model = tf.keras.models.load_model('your_model.h5')  # 또는 'export/1'

# TFLite Converter 사용
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 저장
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

### 2단계 - FastAPI로 TFLite API 서버 구성

In [ ]:
# save as tflite_api_server.py
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import numpy as np
import tensorflow as tf
import cv2
import io

app = FastAPI()

# TFLite Interpreter 로딩
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# 입력/출력 세부 정보
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def preprocess_image(image_bytes):
    img_array = np.frombuffer(image_bytes, np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    img = np.expand_dims(img, axis=0).astype(np.float32)
    return img

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    contents = await file.read()
    img = preprocess_image(contents)

    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    pred_class = int(np.argmax(output_data))

    return JSONResponse({"prediction": pred_class, "scores": output_data.tolist()})